In [ ]:
# @title ## Maintenance Mode On/Off { display-mode: "form" }
#@markdown Set Maintenance-Mode On/Off for this worker (must exist and belong to the API key's owner)
import subprocess
import json
import re
import requests


api_key = "" # @param {type:"string"}
worker_name = "" # @param {type:"string"}

def Set_Maintenance_Mode_function():

    i = False #Worker does not match/exist flag

    command_find_user = f'''curl -X 'GET' \
              'https://stablehorde.net/api/v2/find_user' \
              -H 'accept: application/json' \
              -H 'apikey: {api_key}' \
              -H 'Client-Agent: unknown:0:unknown'
            '''

    find_user_response = subprocess.check_output(command_find_user, shell=True).decode('utf-8')
    data_api_key = json.loads(find_user_response)

    if not ("message" in find_user_response):
        for worker_id in data_api_key['worker_ids']:
            #print(worker_id)

            command_find_worker = f'''curl -X 'GET' \
                      'https://stablehorde.net/api/v2/workers/{worker_id}' \
                      -H 'accept: application/json' \
                      -H 'apikey: {api_key}' \
                      -H 'Client-Agent: unknown:0:unknown'
                    '''
            find_worker_response = subprocess.check_output(command_find_worker, shell=True).decode('utf-8')
            data_worker = json.loads(find_worker_response)
            if (worker_name == data_worker['name']):
                print(worker_id)
                i = True

                worker_uuid = worker_id

                Maintenance_Mode = False # @param {type:"boolean"}
                if (Maintenance_Mode):
                  set_maintenance_mode = "true"
                else:
                  set_maintenance_mode = "false"

                command_maintenance_mode = f'''curl -X 'PUT' \
                  'https://stablehorde.net/api/v2/workers/{worker_id}' \
                  -H 'accept: application/json' \
                  -H 'apikey: {api_key}' \
                  -H 'Client-Agent: unknown:0:unknown' \
                  -H 'Content-Type: application/json' \
                  -d'''+ r''' '{
                  "maintenance":''' + f''' {set_maintenance_mode}
                ''' + r'''}'
                '''


                maintenance_mode_response = subprocess.check_output(command_maintenance_mode, shell=True).decode('utf-8')

                print(maintenance_mode_response)

        if not i:
          print(f'''No worker with the name: {worker_name}, can't change Maintenance Mode''')
          worker_id = 0
    else:
        print(f'''The API key "{api_key}" does not exist''')

Set_Maintenance_Mode_function()